In [9]:
using JuMP
using Gurobi

## Sets

In [10]:
I=2 # 1=gyn og 2=sarkom
U=1 # 1=hast (og 2=ikke-hast)
S=1 # 1 scanner
D=H=5 # 5 dager
R=2 # 2 radiologer
M=5 # 1=tolking intern, 2=tolking ekstern, 3=scanner, 4=MDT, 5=Åpen

;

## Parameters

In [17]:
#i rader, m kolonner.
T_im=[[1 1 1 1 1];
      [1 1 1 1 1]]

G_d=[3 3 3 3 3]

N_sd=[8 8 8 8 8]
L_d=[1 1 1 1 1]

S_sd=[0 0 0 0 1]

D_iuW=[4 4]

K_dr=[[8 8];
      [8 8];
      [8 8];
      [8 8];
      [8 8];]

D_iuD=[0 0]

B_iu=[3 3]

A_id=[[1 0 0 0 0];
      [0 0 1 0 0]]

P=3 # maks antall scannedager en radiolog kan være satt til

E_iud=[[2 2 2 2 2];
       [2 2 2 2 2]]


;

## Variables

In [18]:
m = Model(with_optimizer(Gurobi.Optimizer))
@variable(m, delta_p[1:D] >= 0, Int)
@variable(m, delta_m[1:D] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)
@variable(m, zeta[1:D]>=0, Int)#number of open slots at the scanner at day d
;

Academic license - for non-commercial use only


## Objective function

In [19]:
@objective(m, Min, sum(delta_p+delta_m));

## Constraints

In [20]:
#Constraint2
@constraint(m,[d=1:D], delta_m[d]-delta_p[d] + sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, r=1:R, m=5)==G_d[d])             
                 
#Constraint3
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)+zeta[d]==N_sd[s,d]-S_sd[s,d])
                            
#Constraint4
@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)==D_iuW[i])

#Constraint5
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)==K_dr[d,r])

#Constraint6
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i])

#Constraint7
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#Constraint8
#@constraint(m,[i=1:I,h=1:H, r=1:R], sum(u_i[i,u,d,h,r] for u=1:U,d=1:D if d<=h)==sum(w[i,u,h,r,m] for u=1:U, m=1))                                

#Constraint9*
#@constraint(m,[i=1:I,h=1:H, r=1:R], sum(v[i,u,d,h,r] for u=1:U,d=1:D if d<=h)==sum(w[i,u,h,r,m] for u=1:U, m=2))                                                
                
#Constraint10
#@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)-sum(u_i[i,u,d,h,r] for r=1:R, h=(d:d+B_iu[i]) if h<=D)==0)

#Constraint11*                                                                
#@constraint(m,[i=1:I,u=1:U,d=1:D], sum(v[i,u,d,h,r] for r=1:R, h=(d:d+B_iu[i]) if h<=D)==E_iud[i,d])                                                                                           
 
                                         
                                                
#Constraint12
@constraint(m,[d=1:D], sum(w[i,u,d,r,m] for i=1:I, u=1:U, r=1:R, m=3)==sum(N_sd[s,d] for s=1:S))

#Constraint13
@constraint(m,[d=1:D], sum(beta[d,r] for r=1:R)==L_d[d])
                
#Constraint14
@constraint(m,[d=1:D, r=1:R], K_dr[d,r]*(1-beta[d,r])-sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I, u=1:U, m =1:M if m!=3)==0)

#Constraint15
@constraint(m,[r=1:R], sum(beta[d,r] for d=1:D)<=P)                                                                                  
                                                                                                                                                                
#Constraint16
@constraint(m, [i=1:I,d=1:D], sum(w[i,u,d,r,m] for u=1:U, r=1:R, m=4)==T_im[i,4]*A_id[i,d])
                
#Constraint17
@constraint(m,[i=1:I,d=1:D],sum(gamma[i,d,r] for r=1:R)==A_id[i,d])
                
#Constraint18
@constraint(m,[i=1:I,d=1:D, r=1:R], sum(w[i,u,d,r,m] for u=1:U, m=4)==gamma[i,d,r])               

#Constraint19
@constraint(m, sum(w[i,u,d,r,m] for i=1:I, u=1:U, d=1:D, r=1:R, m=1)==sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))                                                
#Egendefinert. Må sørge for at vi har like mange interne tolkeoppgaver som tildelte slots
                                                                                                                                                

#Constraint20
@constraint(m,[i=1:I,u=1:U,d=1:D], sum(v[i,u,d,h,r] for r=1:R, h=(d+B_iu[i]+1:D) if h<=D)==0)  
                                                                                
#Constraint21*                                                                                
@constraint(m,[i=1:I,u=1:U,d=1:D], sum(u_i[i,u,d,h,r] for r=1:R, h=(d+B_iu[i]+1:D) if h<=D) == 0)
                                                                                                
;                                                                                                
                                                                                                                                                                                                               
#=******ENDRINGER*****
HUSK: Kommenter ut constraints 8-11                                                
Harald: Prøve å få til to indekser i B_iu. Altså B_iu[i,u]. Også noe på E_iud[i,d]=#
                                         
                                                
#Nye constraint10                                                                                                
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i] <= 7
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) == 0)
    else
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) - sum(u_i[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i])%7 if h<d) == 0)                                                                                                                      
    end                                                                                                 
end
                                                                                                                            
#Constraint11*                                                                
#@constraint(m,[i=1:I,u=1:U,d=1:D], sum(v[i,u,d,h,r] for r=1:R, h=(d:d+B_iu[i]) if h<=D)==E_iud[i,d])
#Nye constraint11                                                                                                                            
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i] <= 7
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i] if h<=D) == E_iud[i,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=1:d-1) == 0)                                                                                                            
    else                                                                                                                  
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i] if h<=D) + sum(v[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i])%7 if h<d) == E_iud[i,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R,h=((d+B_iu[i])%7) + 1:(d-1)) == 0)                                                                                                                                        
    end                                                                                                 
end
                                                                                                                                        
#Ny Constraint8
@constraint(m,[i=1:I,h=1:H, r=1:R], sum(u_i[i,u,d,h,r] for u=1:U,d=1:D) == sum(w[i,u,h,r,m] for u=1:U,m=1))

#Ny Constraint9*
@constraint(m,[i=1:I,h=1:H, r=1:R], sum(v[i,u,d,h,r] for u=1:U,d=1:D) == sum(w[i,u,h,r,m] for u=1:U, m=2))                                                                                                                                                  
                                                                                                                

2×5×2 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,3}:
[:, :, 1] =
 v[1,1,1,1,1] + v[1,1,2,1,1] + v[1,1,3,1,1] + v[1,1,4,1,1] + v[1,1,5,1,1] - w[1,1,1,1,2] == 0.0  …  v[1,1,1,5,1] + v[1,1,2,5,1] + v[1,1,3,5,1] + v[1,1,4,5,1] + v[1,1,5,5,1] - w[1,1,5,1,2] == 0.0
 v[2,1,1,1,1] + v[2,1,2,1,1] + v[2,1,3,1,1] + v[2,1,4,1,1] + v[2,1,5,1,1] - w[2,1,1,1,2] == 0.0     v[2,1,1,5,1] + v[2,1,2,5,1] + v[2,1,3,5,1] + v[2,1,4,5,1] + v[2,1,5,5,1] - w[2,1,5,1,2] == 0.0

[:, :, 2] =
 v[1,1,1,1,2] + v[1,1,2,1,2] + v[1,1,3,1,2] + v[1,1,4,1,2] + v[1,1,5,1,2] - w[1,1,1,2,2] == 0.0  …  v[1,1,1,5,2] + v[1,1,2,5,2] + v[1,1,3,5,2] + v[1,1,4,5,2] + v[1,1,5,5,2] - w[1,1,5,2,2] == 0.0
 v[2,1,1,1,2] + v[2,1,2,1,2] + v[2,1,3,1,2] + v[2,1,4,1,2] + v[2,1,5,1,2] - w[2,1,1,2,2] == 0.0     v[2,1,1,5,2] + v[2,1,2,5,2] + v[2,1,3,5,2] + v[2,1,4,5,2] + v[2,1,5,5,2] - w[2,1,5,2,2] == 0.0

## Results

In [21]:
@time begin
    optimize!(m)
    println("Run time: ")
end

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
#println("Dual con1: ", shadow_price(con1[1]))
println("Objective value: ", objective_value(m))

println()
println("****** Variable: x ******")
for u=1:U
    for s=1:S
        for d=1:D
            println()
            println("Day: ",d)
            println("__________________")
            for i=1:I
            if value(x[i,u,s,d]) != 0
                println("x[",i,u,s,d,"] = ", value(x[i,u,s,d]))
            end
                if i==2
                    println("Open slots = ",value(zeta[d]))
                end
            end
        end
    end
end
println()
println("****** Variable: w ******")
for d=1:D
    println()
    println("Day: ",d)
    println("__________________")
    for i=1:I
        #println("Patient group: ", i)
        for u=1:U
            for r=1:R
                #println("Radioligst: ",r)
                for m=1:M
                    if value(w[i,u,d,r,m]) != 0
                        println("w[",i,u,d,r,m,"] = ", value(w[i,u,d,r,m]))
                    end
                end
            end
        end
    end
end
println()
println("****** Variable: u ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(u_i[i,u,d,h,r]) != 0
                        println("u[",i,u,d,h,r,"] = ", value(u_i[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: v ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(v[i,u,d,h,r]) != 0
                        println("v[",i,u,d,h,r,"] = ", value(v[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: delta ******")
for d =1:D
    println()
    println("Day: ",d)
    println("__________________")
    if value(delta_p[d]) !=0
        println("delta_p[",d,"] = ",value(delta_p[d]))
    end
    if value(delta_m[d])!=0
        println("delta_m[",d,"] = ",value(delta_m[d]))
    end
end

println()
println("****** Variable: alpha ******")
for i=1:I
    for u=1:U
        for d=1:D
            if value(alpha[i,u,d]) != 0
                println("alpha[",i,u,d,"] = ", value(alpha[i,u,d]))
            end
        end
    end
end

println()
println("****** Variable: beta ******")
for d=1:D
    for r=1:R
        if value(beta[d,r]) != 0
            println("beta[",d,r,"] = ", value(beta[d,r]))
        end
    end
end

println()
println("****** Variable: gamma ******")
for i=1:I
    for d=1:D
        for r=1:R
            if value(gamma[i,d,r]) != 0
                println("gamma[",i,d,r,"] = ", value(gamma[i,d,r]))
            end
        end
    end
end



Academic license - for non-commercial use only
Optimize a model with 187 rows, 365 columns and 839 nonzeros
Variable types: 0 continuous, 365 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 138 rows and 239 columns
Presolve time: 0.00s
Presolved: 49 rows, 126 columns, 283 nonzeros
Variable types: 0 continuous, 126 integer (5 binary)
Found heuristic solution: objective 9.0000000
Found heuristic solution: objective 7.0000000

Root relaxation: objective 5.000000e+00, 43 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.00000    0    4    7.00000    5.00000  28.6%     -    0s
H    0     0                       5.0000000    5.00000  0.00%     -    0s

Explored 1 nodes (43 simplex iterations) in 0.01 se

## The model

In [22]:
print(m)

Min delta_p[1] + delta_m[1] + delta_p[2] + delta_m[2] + delta_p[3] + delta_m[3] + delta_p[4] + delta_m[4] + delta_p[5] + delta_m[5]
Subject to
 delta_m[1] - delta_p[1] + w[1,1,1,1,5] + w[1,1,1,2,5] + w[2,1,1,1,5] + w[2,1,1,2,5] == 3.0
 delta_m[2] - delta_p[2] + w[1,1,2,1,5] + w[1,1,2,2,5] + w[2,1,2,1,5] + w[2,1,2,2,5] == 3.0
 delta_m[3] - delta_p[3] + w[1,1,3,1,5] + w[1,1,3,2,5] + w[2,1,3,1,5] + w[2,1,3,2,5] == 3.0
 delta_m[4] - delta_p[4] + w[1,1,4,1,5] + w[1,1,4,2,5] + w[2,1,4,1,5] + w[2,1,4,2,5] == 3.0
 delta_m[5] - delta_p[5] + w[1,1,5,1,5] + w[1,1,5,2,5] + w[2,1,5,1,5] + w[2,1,5,2,5] == 3.0
 x[1,1,1,1] + x[2,1,1,1] + zeta[1] == 8.0
 x[1,1,1,2] + x[2,1,1,2] + zeta[2] == 8.0
 x[1,1,1,3] + x[2,1,1,3] + zeta[3] == 8.0
 x[1,1,1,4] + x[2,1,1,4] + zeta[4] == 8.0
 x[1,1,1,5] + x[2,1,1,5] + zeta[5] == 7.0
 x[1,1,1,1] + x[1,1,1,2] + x[1,1,1,3] + x[1,1,1,4] + x[1,1,1,5] == 4.0
 x[2,1,1,1] + x[2,1,1,2] + x[2,1,1,3] + x[2,1,1,4] + x[2,1,1,5] == 4.0
 w[1,1,1,1,1] + w[1,1,1,1,2] + w[1,1,1,1,3] +